In [ ]:
import os
import parser
from pylib.sensor_lib import *
import csv as pycsv
import math
import glob

# 任务列表（如需替换请修改这里）
mission_urls = [
    "/home/wentao/datasets/moments/0a/0ab6c9fd-797f-4bb9-b522-52b0434fe1b1"
]
mission_urls_rec = [os.path.join(url, "rec") for url in mission_urls]

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Store parsed AM results per mission
global_ad_intervals = {}
# Store parsed Base Date per mission 
global_mission_dates = {}
AD_ACTIVE_STATES = [6, 7, 8]
# Control picture 
# If need get picture change ENABLE_AM_PLOT to True
ENABLE_AM_PLOT = True
MAX_PLOT_MISSIONS = 3

os.makedirs("res/AM", exist_ok=True)
if os.path.exists("res/AM"):
    import shutil
    shutil.rmtree("res/AM")
os.makedirs("res/AM", exist_ok=True)

def find_column(df, candidates):
    for col in candidates:
        if col in df.columns:
            return col
    return None

for i, mission_rec in enumerate(mission_urls_rec):
    print(f"\n[{i}] handle AD status: {mission_rec}")
    rec_file = os.path.join(mission_rec, "autonomy_manager_status.rec")

    am_output_dir = f"res/AM/{i}"
    os.makedirs(am_output_dir, exist_ok=True)

    if not os.path.exists(rec_file):
        print(f"WARN! file {rec_file} doesn't exist")
        global_ad_intervals[i] = []
        continue

    # parse rec file
    ret = GeneralParse("autonomy_manager_status_parsing_tool",
                       {"-o": f"{am_output_dir}/"}, rec_file)

    if ret == 0:
        am_csv = f"{am_output_dir}/autonomy_manager_status.csv"

        if os.path.isfile(am_csv):
            if ENABLE_AM_PLOT and i < MAX_PLOT_MISSIONS:
                df_am = pd.read_csv(am_csv)
            else:
                def usecols(c):
                    c_strip = c.strip()
                    base_cols = {
                        "am_status",
                        "autonomy_manager_status",
                        "publish_ts",
                        "timestamp",
                    }
                    return c_strip in base_cols

                df_am = pd.read_csv(am_csv, usecols=usecols)
            df_am.columns = df_am.columns.str.strip()
            status_col = find_column(df_am, ["am_status", "autonomy_manager_status"])
            time_col = find_column(df_am, ["publish_ts", "timestamp"])
            
            if status_col is None:
                print("  Error: no status column found")
                global_ad_intervals[i] = []
                global_mission_dates[i] = "UNKNOWN DATE"
                continue
            if time_col is None:
                print("  Error: no time column found")
                global_ad_intervals[i] = []
                global_mission_dates[i] = "UNKNOWN DATE"
                continue

            print(f"   -> Using status: {status_col}, time: {time_col}")

            status = df_am[status_col].to_numpy()
            ts = df_am[time_col].to_numpy()


            if len(status) == 0:
                print("  WARN: empty AM csv")
                global_ad_intervals[i] = []
                global_mission_dates[i] = "UNKNOWN DATE"
                continue

            active = np.isin(status, AD_ACTIVE_STATES).astype(np.int8)  # 0/1
            diff = np.diff(active)  # +1: 0->1, -1: 1->0

            start_idx = np.where(diff == 1)[0] + 1
            end_idx   = np.where(diff == -1)[0] + 1

            if active[0] == 1:
                start_idx = np.insert(start_idx, 0, 0)
            if active[-1] == 1:
                end_idx = np.append(end_idx, len(active) - 1)
            intervals = [(int(ts[s]), int(ts[e])) for s, e in zip(start_idx, end_idx)]

            global_ad_intervals[i] = intervals
            print(f"extract {len(intervals)} AD active intervals")

            if len(ts) > 0:
                first_dt = pd.to_datetime(ts[0], unit='ns')
                date_str = str(first_dt.date())
                global_mission_dates[i] = date_str
                print(f"   -> Mission Date: {date_str}")
            else:
                global_mission_dates[i] = "UNKNOWN DATE"
            # only store fisrt point date
            if ENABLE_AM_PLOT and i < MAX_PLOT_MISSIONS:
                print("   -> Generating Plot...")
                df_full = pd.read_csv(am_csv)
                df_full.columns = df_full.columns.str.strip()
                
                plot_x = pd.to_datetime(df_full[time_col], unit='ns')

                fig = go.Figure()
                ignore_cols = ["timestamp", "publish_ts", "transmission_time", "plot_time"]
                header = [c for c in df_full.columns if c not in ignore_cols]

                for head in header:
                    fig.add_trace(go.Scatter(
                        x=plot_x,
                        y=df_full[head],
                        name=head
                    ))

                am_state_desp = (
                    "<br />UNKNOWN=0,READY=1,EXIT=2,PREPARE=3,PARTIAL_READY=4,AD_READY=5,<br />"
                    "ENGAGE=6,PARTIAL_RUN=7,RUN=8,DISENGAGE=9,FALLBACK=10,MRC=11"
                )

                fig.update_layout(
                    title=f"Mission {i} AM Status ({global_mission_dates[i]}) - {am_state_desp}",
                    height=400
                )

                #show ad intervals
                ad_intervals_dt = [
                    (pd.to_datetime(s, unit='ns'), pd.to_datetime(e, unit='ns'))
                    for s, e in global_ad_intervals[i]
                ]

                for start_dt, end_dt in ad_intervals_dt:
                    fig.add_vrect(
                        x0=start_dt,
                        x1=end_dt,
                        fillcolor="red",
                        opacity=0.2,
                        layer="below",
                        line_width=0
                    )
                out_html = os.path.join(am_output_dir, "am_status.html")
                fig.write_html(out_html)
                print(f"   -> AM plot saved to {out_html}")
                fig.show()


print("\nall AM missions parsed done.")

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import glob
import pandas as pd
import math
import os
import csv as pycsv
import numpy as np


hardware_resource_urls = [
    os.path.join(mission_url, "system_hardware_resource.rec") 
    for mission_url in mission_urls_rec
]

os.makedirs("res/hhmt", exist_ok=True)
if os.path.exists("res/hhmt"):
    import shutil
    shutil.rmtree("res/hhmt")
os.makedirs("res/hhmt", exist_ok=True)


def filter_stats(data):
    if len(data) == 0: 
        return 0, 0, 0, 0, 0
    average = sum(data) / len(data)
    min_value = min(data)
    max_value = max(data)
    sorted_data = sorted(data)
    max_99 = sorted_data[math.floor(len(sorted_data) * 0.99)]
    max_95 = sorted_data[math.floor(len(sorted_data) * 0.95)]
    return average, min_value, max_value, max_99, max_95

def get_time_column_and_mask(data, current_ad_intervals, skip_rows=600):

    time_mode = "index"
    x_plot = None
    ts_values = None
    
    if "publish_ts" in data.columns:
        time_mode = "publish_ts"
        x_raw = data["publish_ts"].iloc[skip_rows:]
        ts_values = x_raw.values
        x_plot = pd.to_datetime(x_raw, unit='ns')
    elif "timestamp" in data.columns:
        time_mode = "timestamp"
        x_raw = data["timestamp"].iloc[skip_rows:]
        ts_values = x_raw.values
        x_plot = pd.to_datetime(x_raw, unit='ns')
    else:
        time_mode = "index"
        x_plot = np.arange(skip_rows, len(data))
    
    is_ad_mask = np.zeros(len(x_plot), dtype=bool)
    if time_mode != "index" and len(current_ad_intervals) > 0 and ts_values is not None:
        for start, end in current_ad_intervals:
            is_ad_mask |= (ts_values >= start) & (ts_values <= end)
    
    return time_mode, x_plot, ts_values, is_ad_mask

def draw_ad_intervals(ax, current_ad_intervals, x_plot, current_date):
    if len(current_ad_intervals) == 0 or x_plot is None or len(x_plot) == 0:
        return
    
    x_min = x_plot.min() if hasattr(x_plot, 'min') else min(x_plot)
    x_max = x_plot.max() if hasattr(x_plot, 'max') else max(x_plot)
    
    for start, end in current_ad_intervals:
        start_dt = pd.to_datetime(start, unit='ns')
        end_dt = pd.to_datetime(end, unit='ns')
        
        if end_dt < x_min or start_dt > x_max:
            continue
        
        draw_s = max(start_dt, x_min)
        draw_e = min(end_dt, x_max)
        if draw_e > draw_s:
            ax.axvspan(draw_s, draw_e, color='red', alpha=0.15)

# main loop
for i in range(len(hardware_resource_urls)):
    current_ad_intervals = global_ad_intervals.get(i, [])
    current_date = global_mission_dates.get(i)
    output_dir = f"res/hhmt/xeon_a/{i}"
    major_chip = GeneralParse("hardware_health_monitor_tool", {"f": output_dir, "c": "xeon_a"}, hardware_resource_urls[i])
    
    if major_chip == 0:
        major_chip_list = glob.glob(f"{output_dir}/cpu_process/*")
        
        # A. CPU UTIL statistics
        cpu_long_path = f"res/hhmt/cpu_long_{i}.csv"
        if os.path.exists(cpu_long_path):
            os.remove(cpu_long_path)

        
        t_a = t_mi = t_mx = t_ad_a = t_ad_mx = t_man_a = t_man_mx = 0
        
        for csv in major_chip_list:
            data = pd.read_csv(csv, encoding='ISO-8859-1')
            data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")
            
            if "cpu_util" not in data.columns or len(data["cpu_util"]) < 600:
                continue
            
            y = data["cpu_util"].iloc[600:]
            
            time_mode, x_plot, ts_values, is_ad_mask = get_time_column_and_mask(
                data, current_ad_intervals, skip_rows=600
            )
            
            y_ad = y[is_ad_mask]      # AD 
            y_manual = y[~is_ad_mask] # no AD

            plt.figure(figsize=(12, 4), dpi=200)
            plt.title(f"CPU Util - {os.path.basename(csv)}")
            plt.plot(x_plot, y, label=csv.split("/")[-1], linewidth=0.5)

            if time_mode != "index":
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                plt.xlabel(current_date)
                plt.ylabel("CPU Util (%)")

                draw_ad_intervals(plt.gca(), current_ad_intervals, x_plot, current_date)
            else:
                plt.xlabel("Index Count")

            plt.legend()

            # statistics
            a, mi, mx, mx99, mx95 = filter_stats(y.values)
            ad_a, ad_mi, ad_mx, ad_99, ad_95 = filter_stats(y_ad.values)
            man_a, man_mi, man_mx, man_99, man_95 = filter_stats(y_manual.values)

            t_a += a; t_mx += mx; t_ad_a += ad_a; t_ad_mx += ad_mx
            t_man_a += man_a; t_man_mx += man_mx

            write_header = not os.path.exists(cpu_long_path)
            with open(cpu_long_path, "a", newline='') as f:
                writer = pycsv.writer(f)
                if write_header:
                    writer.writerow([
                        "filename", 
                        "AD_Avg", "AD_95", "AD_99", 
                        "MAN_Avg", "MAN_95", "MAN_99"
                    ])
                with open(cpu_long_path, "a", newline='') as f:
                    writer = pycsv.writer(f)
                    if write_header:
                        writer.writerow(["filename", "AD_Avg", "AD_95", "AD_99", "MAN_Avg", "MAN_95", "MAN_99"])

                    # ad_a, ad_95, ad_99  man_a, man_95, man_99
                    writer.writerow([
                        os.path.basename(csv),
                        f"{ad_a:.2f}", ad_95, ad_99,
                        f"{man_a:.2f}", man_95, man_99
                    ])

            
        with open(cpu_long_path, "a", newline='') as f:
            writer = pycsv.writer(f)
            writer.writerow([
                "TOTAL_SUM_AVG", "AD",
                t_ad_a, "", t_ad_mx, "", ""
            ])
            writer.writerow([
                "TOTAL_SUM_AVG", "MAN",
                t_man_a, "", t_man_mx, "", ""
            ])
        # B. RAM UTIL statistics
        ram_long_path = f"res/hhmt/ram_long_{i}.csv"
        if os.path.exists(ram_long_path):
            os.remove(ram_long_path)

        
        
        t_ad_a = t_ad_mx = 0
        t_man_a = t_man_mx = 0
        
        for csv in major_chip_list:
            data = pd.read_csv(csv, encoding='ISO-8859-1')
            data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")

            ram_cols = [c for c in data.columns if "ram" in c and "util" in c]
            if not ram_cols or "ram_util" not in data.columns or len(data["ram_util"]) < 600:
                continue

            y = data["ram_util"].iloc[600:]
            
            time_mode, x_plot, ts_values, is_ad_mask = get_time_column_and_mask(
                data, current_ad_intervals, skip_rows=600
            )
            
            y_ad = y[is_ad_mask]
            y_manual = y[~is_ad_mask]

            plt.figure(figsize=(12, 4), dpi=200)
            plt.title(f"RAM Util(MB) - {os.path.basename(csv)}")
            plt.plot(x_plot, y/1024/1024, label=csv.split("/")[-1], linewidth=0.5)
            
            if time_mode != "index":
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                plt.xlabel(current_date)
                plt.ylabel("RAM Util (MB)")
                draw_ad_intervals(plt.gca(), current_ad_intervals, x_plot, current_date)
            else:
                plt.xlabel("Index Count")

            plt.legend()

            ad_a, ad_mi, ad_mx, ad_99, ad_95 = filter_stats(y_ad.values / 1024 / 1024)
            man_a, man_mi, man_mx, man_99, man_95 = filter_stats(y_manual.values / 1024 / 1024)

            t_ad_a += ad_a; t_ad_mx += ad_mx
            t_man_a += man_a; t_man_mx += man_mx

            write_header = not os.path.exists(ram_long_path)
            with open(ram_long_path, "a", newline="") as f:
                writer = pycsv.writer(f)
                if write_header:
                    writer.writerow(["filename", "AD_Avg", "AD_95", "AD_99", "MAN_Avg", "MAN_95", "MAN_99"])

                writer.writerow([
                    os.path.basename(csv),
                    f"{ad_a:.2f}", ad_95, ad_99,
                    f"{man_a:.2f}", man_95, man_99
                ])
        

            plt.tight_layout()
            plt.savefig(f"res/hhmt/xeon_a/cpu_process_ram_util_{i}.png")
            plt.close()
        with open(ram_long_path, "a", newline="") as f:
          writer = pycsv.writer(f)
          writer.writerow(["TOTAL_SUM_AVG", "AD", t_ad_a, t_ad_mx])
          writer.writerow(["TOTAL_SUM_AVG", "MAN", t_man_a, t_man_mx])


print("Final Fix: Both CPU and RAM charts use consistent absolute time formatting.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

interested_keys = ['us', 'ni', 'sy', 'id', 'wa', 'hi', 'si', 'st']
# add interested cpu cores here, up to cpu15 for 2160
cpus_major_chip = ["cpu0", "cpu1", "cpu2", "cpu3", "cpu4", "cpu5"]
chip_txt_map = {
    "major_chip": "cpu_stat_major_chip.txt",
    "j3": "cpu_stat_j3.txt",
    "j5_a": "cpu_stat_j5a.txt",
    "j5_b": "cpu_stat_j5b.txt"
}
k_keys = ['us', 'ni', 'sy', 'id', 'wa', 'hi', 'si', 'st']
k_timestamp = 'timestamp'

core_cpu_stat_url = [
    os.path.join(mission_url, "cpu_stat_raw_data.rec") 
    for mission_url in mission_urls_rec
]

os.makedirs("res/cpu_stat", exist_ok=True)
if os.path.exists("res/cpu_stat"):
    import shutil
    shutil.rmtree("res/cpu_stat")
os.makedirs("res/cpu_stat", exist_ok=True)
#dict: cpu, cpu0 ... cpuN, timestamp -> [n1, n2, ...]
prev_sample = {}
def cpu_stat_line_reader(file):
    global prev_sample
    line = file.readline()
    if not line:
        return 'EOF'
    if len(line) == 0:
        return None
    # cpu0 4799603 5603 6599370 142116234 702888 0 1710827 0 0 0
    sample = line.split()
    if len(sample) == 0:
        return None
    cpu_x = sample[0]
    if cpu_x[0:3] != 'cpu' and cpu_x != k_timestamp:
        return None
    if not cpu_x in prev_sample:
        prev_sample[cpu_x] = sample
        return None
    if cpu_x == k_timestamp:
        return [cpu_x, int(sample[1])]

    result = [int(sample[j]) - int(prev_sample[cpu_x][j]) for j in range(1, len(sample))]
    cpu_total = sum(result)

    if cpu_total == 0:
      utilization = [0.0] * min(len(k_keys),len(sample) - 1)
    else:
      utilization = [float(result[i]) / cpu_total * 100 for i in range(0,min(len(k_keys),len(sample) - 1))]


    prev_sample[cpu_x] = sample
    return [cpu_x] + utilization

# Read a page of cpu stat, e.g.
# cpu  65665104 76955 100106908 1678843958 6257365 0 2154387 0 0 0
# cpu0 4799603 5603 6599370 142116234 702888 0 1710827 0 0 0
# cpu1 6109061 6224 10094859 137375073 635412 0 292563 0 0 0
# cpu2 5605250 6527 9006099 139014225 547225 0 85332 0 0 0
# cpu3 5919762 6152 9722107 138148694 422704 0 23636 0 0 0
# timestamp 1630087906722078194 
def cpu_stat_page(file):
    page = {}
    while True:
        result = cpu_stat_line_reader(file)
        if result is None:
            continue
        if result == 'EOF':
            break
        page[result[0]] = result
        if result[0] == k_timestamp:
            break
    return page

def cpu_stat_reader(cpu_stat, keys):
    # Map 'us':0, 'ni':1, ...
    global prev_sample
    prev_sample = {}

    local_k_keys = list(k_keys)

    k_keys_map = {key:i for i,key in enumerate(local_k_keys)}

    timestamp = []
    # For examples, [user, sys, idle], 
    cpu_util = [{} for i in range(len(keys))]
    # Select items (indices) from k_keys[]
    sel = [k_keys_map[keys[i]] for i in range(len(keys))]
    with open(cpu_stat, 'r') as cpu_stat_file:
        while True:
            page = cpu_stat_page(cpu_stat_file)
            if len(page) == 0:
                break
            for k,v in page.items():
                if k == k_timestamp:
                    timestamp.append(v[1]) #keep ns precise
                else:
                    cpu = k
                    for i in range(len(sel)):
                        if not cpu in cpu_util[i]:
                            cpu_util[i][cpu] = []
                        cpu_util[i][cpu].append(v[sel[i] + 1])
    if "id" in keys:
        # in the end add  1-idl
        sel.append(len(k_keys))
        local_k_keys.append("1-id")
        idx_id = k_keys_map["id"]
        for cpu in cpus_major_chip:
            if cpu not in cpu_util[idx_id]:
                continue
            # Ensure cpu_util has enough sublists for the new key
            while len(cpu_util) <= len(sel) - 1:
                cpu_util.append({})
            cpu_util[len(sel)-1][cpu] = [ 100 - x for x in cpu_util[idx_id][cpu]]

    return timestamp, cpu_util, sel

def analyisi_cpu_stat(data, reverse_flag=False):
    if len(data) == 0:
        return 0, 0, 0, 0, 0
    
    average = sum(data) / len(data)
    min_value = min(data)
    max_value = max(data)
    sorted_data = sorted(data)
    
    n = len(sorted_data)
    if reverse_flag:
        idx_99 = max(0, math.floor(n * 0.01))
        idx_95 = max(0, math.floor(n * 0.05))
        max_99 = sorted_data[idx_99]
        max_95 = sorted_data[idx_95]
    else:
        idx_99 = min(n - 1, math.floor(n * 0.99))
        idx_95 = min(n - 1, math.floor(n * 0.95))
        max_99 = sorted_data[idx_99]
        max_95 = sorted_data[idx_95]
        
    return average, min_value, max_value, max_99, max_95



def draw_cpu_stat_plots(chip_name, keys):
    dir_path = "res/cpu_stat/"
    
    # 检查目录是否存在
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        return

    for mission_dir in os.listdir(dir_path):
        current_path = os.path.join(dir_path, mission_dir)
        if not os.path.isdir(current_path): continue
        
        g_ad_intervals = globals().get("global_ad_intervals", {})
        if not isinstance(g_ad_intervals, dict): g_ad_intervals = {} # 确保它是字典
        
        try:
            mission_idx = int(mission_dir)
            current_ad_intervals = g_ad_intervals.get(mission_idx, [])
        except:
            mission_idx = -1
            current_ad_intervals = []
            
        file_path = os.path.join(current_path, chip_txt_map.get(chip_name, ""))
        if not os.path.exists(file_path) or os.path.getsize(file_path) < 100:
            continue
            
        print(f"Plotting Core Stat for Mission {mission_dir}...")
        
        # 2. Read data
        raw_timestamps, cpu_util, sel = cpu_stat_reader(file_path, keys)
        
        if len(raw_timestamps) < 600: continue
        
        # 3. Prepare Time Axis
        t_slice = raw_timestamps[600:]
        x_plot = pd.to_datetime(t_slice, unit='ns')
        ts_values = np.array(t_slice) 

        is_ad_mask = np.zeros(len(ts_values), dtype=bool)
        if len(current_ad_intervals) > 0:
            for start, end in current_ad_intervals:
                is_ad_mask |= (ts_values >= start) & (ts_values <= end)

        # 4. Start plotting for each CPU core
        for cpu in cpus_major_chip:
            stats_csv_path = f"{current_path}/{chip_name}_{cpu}_stats.csv"
            if os.path.exists(stats_csv_path): 
                os.remove(stats_csv_path)

            with open(stats_csv_path, 'w', newline='') as f:
                writer = pycsv.writer(f)
                writer.writerow([
                    "core_index", 
                    "AD_Avg", "AD_Min", "AD_Max", "AD_99", "AD_95",
                    "MAN_Avg", "MAN_Min", "MAN_Max", "MAN_99", "MAN_95"
                ])  

            plt.clf()
            plt.figure(figsize=(16, 6), dpi=150)
            
            has_data = False
            for i in range(len(sel)):
                sel_key = k_keys[sel[i]] if sel[i] < len(k_keys) else "1-id"
                
                if cpu not in cpu_util[i] or len(cpu_util[i][cpu]) < 600:
                    continue
                
                y_data = np.array(cpu_util[i][cpu][600:])
                
                min_len = min(len(y_data), len(is_ad_mask))
                y_data_aligned = y_data[:min_len]
                mask_aligned = is_ad_mask[:min_len]
                
                y_ad = y_data_aligned[mask_aligned]
                y_manual = y_data_aligned[~mask_aligned]

                # Analyze statistics
                ad_a, ad_mi, ad_mx, ad_99, ad_95 = analyisi_cpu_stat(y_ad, sel_key=="id")
                man_a, man_mi, man_mx, man_99, man_95 = analyisi_cpu_stat(y_manual, sel_key=="id")

                full_key_name = f"{chip_name}_{cpu}_{sel_key}"
                # write to CSV
                with open(stats_csv_path, 'a', newline='') as f:
                    writer = pycsv.writer(f)
                    writer.writerow([
                        full_key_name, 
                        f"{ad_a:.2f}", ad_mi, ad_mx, ad_99, ad_95,
                        f"{man_a:.2f}", man_mi, man_mx, man_99, man_95
                    ])
                if sel_key == "id": continue 
                
                # Plot line
                min_len = min(len(x_plot), len(y_data))
                plt.plot(x_plot[:min_len], y_data[:min_len], label=sel_key, linewidth=0.8)
                has_data = True

            # 5. Draw red background and labels
            if has_data:
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                
                xlabel_text = f"Mission {mission_dir}" 
                
                # 尝试获取全局变量
                g_dates = globals().get("global_mission_dates")
                if isinstance(g_dates, dict):
                    xlabel_text = g_dates.get(mission_idx, xlabel_text)
                
                plt.xlabel(xlabel_text)
                plt.ylabel("CPU Utilization (%)")
                
                for start, end in current_ad_intervals:
                    start_dt = pd.to_datetime(start, unit='ns')
                    end_dt = pd.to_datetime(end, unit='ns')
                    if end_dt < x_plot.min() or start_dt > x_plot.max(): continue
                    draw_s = max(start_dt, x_plot.min())
                    draw_e = min(end_dt, x_plot.max())
                    if draw_e > draw_s:
                        plt.axvspan(draw_s, draw_e, color='red', alpha=0.15)

                plt.title(f"Mission {mission_dir} - {chip_name} {cpu}")
                plt.legend(loc='upper right')
                plt.tight_layout()
                plt.savefig(f"{current_path}/{chip_name}_{cpu}_plot.png")
            
            plt.close()
ret_major_chip = 0
for i in range(len(core_cpu_stat_url)):
    output_dir = "res/cpu_stat/%d" % i
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    ret_major_chip = GeneralParse("io_packet_to_txt_tool", {"o": f"{output_dir}/{chip_txt_map['major_chip']}", "c": "xeon_a"}, core_cpu_stat_url[i])
    if (ret_major_chip != 0):
        break
if (ret_major_chip == 0):
    draw_cpu_stat_plots("major_chip", interested_keys)